构建train、dev、test.txt

In [1]:
import pandas as pd
import random 

In [2]:
file_name = './company_event/Event_new_10.22.xlsx'
data = pd.read_excel(file_name,engine="openpyxl")

In [3]:
data.head()
data_len = data.shape[0] - 10

In [4]:
data_len

373299

In [5]:
all_company = set(list(data['CompanyName']))
all_event = set(list(data['Event_Level1']))
all_event_cat = set(list(data['Event_Cat']))
all_datasets = []

In [6]:
cas_file_name = './company_event/casuality.xlsx'
cas_data = pd.read_excel(cas_file_name,engine="openpyxl")

In [7]:
cas_data.head()

""


In [8]:
cas_dic = {}
for row in cas_data.itertuples():
    if row[1] not in cas_dic:
        cas_dic[row[1]] = []
    cas_dic[row[1]].append(row[2])

In [9]:
for company in all_company:
    
    data_c = data[data['CompanyName'].str.contains(company)] #筛选
    try:
        data_c = data_c.sort_values(by='EventTime', ascending=True) #时间排序
    except:
        print(company + "time error")
    data_c = data_c.drop_duplicates(subset = [ 'EventTime' ], keep = 'first') #事件时间去重
    if(data_c.shape[0]<15):
        continue
        
    #print(company+str(data_c.shape[0]))
    
    x_list = []
    y_list = []
    true_list = []
    flag = -1
    
    dex = 0
    for row in data_c.itertuples():
        x_ = []
        y_ = []
        cas_x = []
        cas_dex = 0
        dex+=1
        if(dex>=12):
            break
        
        if(dex<9):
            x_.append(getattr(row, 'Event_Cat').split("###")[0])
            x_.append('None')
            x_.append("发生")
            x_.append(getattr(row, 'Event_Level1'))
            x_.append(getattr(row, 'CompanyName'))
            x_.append("客体公司")
            cas_x.append(str(getattr(row, 'Event_Level1')))
            x_list.append(tuple(x_))
        else:# 9 10 11  3个事件    
            y_.append(getattr(row, 'Event_Cat').split("###")[0])
            y_.append('None')
            y_.append("发生")
            y_.append(getattr(row, 'Event_Level1'))
            y_.append(getattr(row, 'CompanyName'))
            y_.append("客体公司")
            y_list.append(tuple(y_))
            if(dex==9):
                true_list = y_
    
    eff_event = []
    for line in cas_x:
        eff_event += cas_data[line]
    eff_event = list(set(eff_event))
    
    y_ = []
    r1 = random.randint(1,data_len-10)
    #print(random.randint(1,data_len))
    
    
    if len(eff_event)!= 0:
        current_event = str(eff_event[random.randint(1,len(eff_event))])
        y_.append(data.iloc[r1]['Event_Cat'].split("###")[0])
        y_.append('None')
        y_.append("发生")
        #y_.append(data.iloc[r1]['Event_Level1'])
        y_.append(current_event)
        y_.append(data.iloc[r1]['CompanyName'])
        y_.append("客体公司")
        y_list.append(tuple(y_))
    else:
        y_.append(data.iloc[r1]['Event_Cat'].split("###")[0])
        y_.append('None')
        y_.append("发生")
        y_.append(data.iloc[r1]['Event_Level1'])
        y_.append(data.iloc[r1]['CompanyName'])
        y_.append("客体公司")
        y_list.append(tuple(y_))
    

    y_ = []
    r2 = random.randint(1,data_len-10)
    if len(eff_event)!= 0:
        current_event = str(eff_event[random.randint(1,len(eff_event))])
        y_.append(data.iloc[r1]['Event_Cat'].split("###")[0])
        y_.append('None')
        y_.append("发生")
        #y_.append(data.iloc[r2]['Event_Level1'])
        y_.append(current_event)
        y_.append(data.iloc[r2]['CompanyName'])
        y_.append("客体公司")
        y_list.append(tuple(y_))
    else:
        y_.append(data.iloc[r1]['Event_Cat'].split("###")[0])
        y_.append('None')
        y_.append("发生")
        y_.append(data.iloc[r2]['Event_Level1'])
        y_.append(data.iloc[r2]['CompanyName'])
        y_.append("客体公司")
        y_list.append(tuple(y_))
        
    random.shuffle(y_list)
    
    for index,line in enumerate(y_list):
        if(line==tuple(true_list)):
            flag = index
    
    one_data = []
    one_data.append(x_list)
    one_data.append(y_list)
    one_data.append(flag)

    all_datasets.append(one_data)

In [10]:
all_datasets[135]

[[('亏损', 'None', '发生', '业务异常', '南大光电', '客体公司'),
  ('处罚', 'None', '发生', '违法违规', '南大光电', '客体公司'),
  ('同比下降', 'None', '发生', '业务异常', '南大光电', '客体公司'),
  ('低于', 'None', '发生', '经营异常', '南大光电', '客体公司'),
  ('低于', 'None', '发生', '经营异常', '南大光电', '客体公司'),
  ('利润下降', 'None', '发生', '经营异常', '南大光电', '客体公司'),
  ('充公', 'None', '发生', '违法违规', '南大光电', '客体公司'),
  ('同比下降', 'None', '发生', '业务异常', '南大光电', '客体公司')],
 [('同比下降', 'None', '发生', '业务异常', '南大光电', '客体公司'),
  ('亏损', 'None', '发生', '业务异常', '正元智慧', '客体公司'),
  ('超标', 'None', '发生', '经营异常', '南大光电', '客体公司'),
  ('亏损', 'None', '发生', '违法违规', '三一重工', '客体公司'),
  ('同比下降', 'None', '发生', '业务异常', '南大光电', '客体公司')],
 4]

In [11]:
import pickle
def pickleFile(db,file):
    with open(file,"w",encoding='utf-8') as dbFile:
        pickle.dump(db,dbFile)
 
def unpickeFile(file):
    with open(file,"rb") as dbFile:
        return pickle.load(dbFile)

In [12]:
train_file = './company_event/train.txt'
dev_file = './company_event/dev.txt'
test_file = './company_event/test.txt'
random.shuffle(all_datasets)

wf = open(train_file,'wb')
pickle.dump(all_datasets[0:1600],wf)

wf = open(dev_file,'wb')
pickle.dump(all_datasets[1600:2100],wf)

wf = open(test_file,'wb')
pickle.dump(all_datasets[2100:2340],wf)

aaa = []
wf = open('dev_index.pickle','wb')
pickle.dump(aaa,wf)

训练词向量

In [80]:
sen_line = []
with open("2007.txt","r",encoding='utf-8') as f:
    for line in f:
        sen_line.append(line.split("###")[1].replace("\u3000",""))
        
with open("2008.txt","r",encoding='utf-8') as f:
    for line in f:
        sen_line.append(line.split("###")[1].replace("\u3000",""))
        
with open("2009.txt","r",encoding='utf-8') as f:
    for line in f:
        sen_line.append(line.split("###")[1].replace("\u3000",""))
        
with open("sentences.txt","w",encoding='utf-8') as f:
    for line in sen_line:
        f.write(line+"\n")
        
    for line in data['Sentences']:
        f.write(line+"\n")

分词部分

In [15]:
#保留词
# all_company = set(list(data['CompanyName']))
# all_event = set(list(data['Event_Level1']))
# all_event_cat = set(list(data['Event_Cat']))
with open('no_cut.txt',"w",encoding='utf-8') as f:
    for line in all_company:
        f.write(line+"\n")
        
    for line in list(set(all_event)):
        f.write(line+"\n")
        
    for line in list(set(all_event_cat)):
        f.write(line.split("###")[0]+"\n")
        f.write(line.split("###")[0]+"_发生\n")
        
    f.write("None")

In [81]:
import logging
import multiprocessing
import os.path
import sys
import jieba

from gensim.models import Word2Vec
from gensim.models.word2vec import PathLineSentences

In [82]:
jieba.load_userdict("no_cut.txt")

In [83]:
filePath = 'sentences.txt'
fileSegWordDonePath = 'cut_sentences.txt'



def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

# 打印中文列表
def PrintListChinese(list):
    pass

# 读取文件内容到列表
fileTrainRead = []  
with open(filePath, 'r',encoding='utf-8') as fileTrainRaw:
    for line in fileTrainRaw:  # 按行读取文件
        fileTrainRead.append(line)

# jieba分词后保存在列表中
fileTrainSeg = []
for i in range(len(fileTrainRead)):
    fileTrainSeg.append([' '.join(list(jieba.cut(fileTrainRead[i][9:-11], cut_all=False)))])
    if i % 100 == 0:
        pass
        #print(i)


stopwords = stopwordslist('stopwords.txt')  # 这里加载停用词的路径
for i in fileTrainRead:   
    sentence_seged = jieba.cut(i.strip())
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                fileTrainSeg.append(word)


        
# 保存分词结果到文件中
with open(fileSegWordDonePath, 'w', encoding='utf-8') as fW:
    for i in range(len(fileTrainSeg)):
        fW.write(fileTrainSeg[i][0])
        fW.write('\n')

"""
gensim word2vec获取词向量
"""

import warnings
import logging
import os.path
import sys
import multiprocessing

import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

# 忽略警告
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')



# inp为输入语料, outp1为输出模型, outp2为vector格式的模型
inp = fileSegWordDonePath
out_model = 'word2vector_1.model'
out_vector = 'word2vector_1.vector'

# 训练skip-gram模型
model = Word2Vec(LineSentence(inp), size=128, window=5, min_count=5,
                 workers=multiprocessing.cpu_count())

# 保存模型
model.save(out_model)
# 保存词向量
model.wv.save_word2vec_format(out_vector, binary=False)

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL

KeyboardInterrupt



id-word 映射

In [89]:
id_word = []
with open('word2vector_1.vector',"r",encoding='utf-8') as f:
    dex = -1
    for line in f:
        id_word.append(str(dex)+" "+line.split( )[0])
        dex+=1
        
from pandas.core.frame import DataFrame
id_word = id_word[1:]
data_=DataFrame({'index':id_word})

data_.to_csv('id_word.csv',sep=',', header=True, index=False,encoding='utf-8')

word2vector word-id

In [58]:
word_id = {}
for line in id_word:
    line = str(line)
    word = line.split( )[1]
    idd = line.split( )[0]
    word_id[word] = idd
new_word2vector = []
with open('word2vector_1.vector',"r",encoding='utf-8') as f:
    dex = 0
    for line in f:
        dex+=1
        if(dex!=1):
            new_word2vector.append(  str( word_id[line.split( )[0]] ) + " " +  " ".join(line.split( )[1:]) )
            
with open('new_word2vector.vector',"w",encoding='utf-8') as f:
    for line in new_word2vector:
        f.write(line+"\n")

构建输入的事件转移图data2

In [27]:
dict_data2 = {}
for data_one in all_datasets:
    context = data_one[0]
    choice = data_one[1]
    for dex in range(len(context)-1):
        if context[dex][0] not in dict_data2:
            dict_data2[context[dex][0]] = {}
        #print(dict_data2)
        if context[dex+1][0] not in dict_data2[context[dex][0]]:
            dict_data2[context[dex][0]][context[dex+1][0]] = 0
        dict_data2[context[dex][0]][context[dex+1][0]] += 1
        
    for dex in range(len(choice)-1):
        if choice[dex][0] not in dict_data2:
            dict_data2[choice[dex][0]] = {}
        
        if choice[dex+1][0] not in dict_data2[choice[dex][0]]:
            dict_data2[choice[dex][0]][choice[dex+1][0]] = 0
        dict_data2[choice[dex][0]][choice[dex+1][0]] += 1
        
    
static_data2 = {}
for v1, line in dict_data2.items():
    #一个v1
    sum_ = 0
    for v2, num in line.items():
        #print(line)
        if(str(v1)!=str(v2)):
            sum_ += int(num)
    
    for v2, num in line.items():
        if v1 not in static_data2:
            static_data2[v1] = {}
        static_data2[v1][v2] = float(dict_data2[v1][v2]/sum_)
        
data_line = []
for v1, line in static_data2.items():
    for v2, num_ in line.items():
        s1 = 0
        s2 = 0
        #if v1 in word_id:
        #    s1 = word_id[v1]
        #if v2 in word_id:
        #    s2 = word_id[v2]
        #data_line.append(str(s1)+" "+str(s2)+" "+str(static_data2[v1][v2]))
        line_ = []
        line_.append(v1)
        line_.append(v2)
        line_.append(static_data2[v1][v2])
        data_line.append(line_)

In [28]:
import csv 
with open('event_graph_.csv', 'w', newline='',encoding='utf-8') as csvfile:
    writer  = csv.writer(csvfile)
    for row in data_line:
        writer.writerow(row)

In [88]:
with open("my_data2.csv","w",encoding="utf-8") as f:
    for line in data_line:
        f.write(line+"\n")

In [91]:
node_list_map={}
node_list =  [1,2,3,4,5]
for i in node_list:
    node_list_map[i]=len(node_list_map)

In [92]:
node_list_map

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

In [6]:
file1 = 'train.txt'

In [7]:
import pickle

In [8]:
file = open(file1,"rb")

In [9]:
train_list = pickle.load(file)

In [16]:
with open("train_wb.txt","w",encoding='utf-8') as f:
    for line in train_list:
        f.write(str(line)+"\n")

In [41]:
a = {'1':2,'3':4}

In [46]:
for keys,value in a.items():
        print(keys,value)

1 2
3 4


In [1]:
#define f(x) x*x

整理触发词

In [1]:
import pandas as pd
import random 

In [2]:
file_name = 'Event_new_10.22.xlsx'
data = pd.read_excel(file_name)

In [14]:
all_event = data['Event_Level3']
all_event_cat = data['Event_Cat']
all_datasets = {}

In [15]:
for i in range(len(all_event)):
    if all_event[i] not in all_datasets:
        all_datasets[all_event[i]] = []
    else:
        if all_event_cat[i] not in all_datasets[all_event[i]]:
            all_datasets[all_event[i]].append(all_event_cat[i])

In [19]:
all_datasets

{'业绩下滑': ['亏损',
  '同比下降',
  '产能过剩',
  '利润下滑',
  '盈利能力下降',
  '人民币升值',
  '净利润下降',
  '负收益',
  '原材料价格上涨',
  '限产',
  '不达预期',
  '公司业绩下滑',
  '收益下降',
  '盈利下滑',
  '资产减值',
  '毛利率下降',
  '利润减少',
  '盈利下降',
  '成本上涨',
  '股票减持',
  '公司业绩下降',
  '收入下滑',
  '产量减少',
  '需求下降',
  '收益下滑',
  '资产减持',
  '销售费用增加',
  '存货增加',
  '收益为负',
  '销量减少',
  '薪酬上涨',
  '售价降低',
  '收益缩水',
  '公司业绩负增长',
  '交易量减少',
  '外币贬值',
  '盈利负增长',
  '盈利萎缩',
  '公司业绩缩水',
  '收益萎缩',
  '收益负增长'],
 '调查': ['立案', '调查'],
 '被举报': ['举报', '投诉', '揭发', '检举', '告发'],
 '行政处罚': ['罚款',
  '限期整改',
  '停产整改',
  '制裁',
  '充公',
  '处罚',
  '罚金',
  '重罚',
  '责令改正',
  '整顿',
  '警告',
  '通报批评',
  '行政处罚',
  '整治',
  '罚没',
  '责令整改',
  '责令停产',
  '禁入',
  '吊销',
  '没收',
  '扣押',
  '暂扣',
  '行政拘留',
  '责令停业',
  '责令立即改正',
  '查封扣押'],
 '传闻调查': ['消息称###调查',
  '传言###调查',
  '传闻###调查',
  '盛传###调查',
  '谣言###调查',
  '传言###立案',
  '爆料###调查',
  '消息称###立案',
  '言论###调查',
  '流传###调查',
  '风传###调查',
  '传闻###立案',
  '谣传###立案',
  '谣传###调查',
  '谣言###立案',
  '爆料###立案',
  '流传###立案'],
 '食品添加剂': ['低于',
  '不合格',
  '缺

In [29]:
with open("catrigger.txt","w",encoding='utf-8') as f:
    for index,item in all_datasets.items():
        f.write(str(index)+"---"+str(item)+"\n")